# LOWESS filtering


A wrapper needs to be written to accept pandas object and make sure input time is sorted

In [4]:
import os
from glob import glob

import numpy as np
import pandas as pd
import xarray as xr

%matplotlib inline
import matplotlib.pyplot as plt
import hvplot.pandas  # noqa
import holoviews as hv

#
import pynsitu as pin
from pynsitu.maps import crs

from lib import raw_dir, root_dir, images_dir, KEYS, color, columns

from numba import njit, guvectorize, int32, float64, prange

from pynsitu.geo import GeoAccessor, compute_velocities, compute_accelerations

In [5]:
key = KEYS[0]
df = pd.read_csv(glob(os.path.join(raw_dir, 'L1_' + key+'*'))[0], parse_dates = ['time'], dtype={"id":str}).set_index('id')
df_ = df.loc['0-4388553'].sort_values('time').dropna()
df_['date'] = (df_.time-df_.time.min())/pd.Timedelta('1s')

In [22]:
@njit
def advance_search(nt, time, t, i, delta_plus, delta_minus):
    """ find next closest neighbourgh by searching in positive and negative 
    directions with respect to index i and update delta_minus, delta_plus
    """
    # delta=0 means search as stopped in that direction
    #  compute distances with i+delta_plus and i+delta_minus points
    if delta_plus>0 and i+delta_plus<nt:
        d_plus = abs(time[i+delta_plus]-t)
    else:
        d_plus = -1.
    if delta_minus<0 and i+delta_minus>=0:
        d_minus = abs(time[i+delta_minus]-t)
    else:
        d_minus = -1.
    # update delta_plus or delta_minus
    if d_minus!=-1 and (d_plus==-1 or d_minus<=d_plus):#correction cas d_plus=d_minus
        i_next = i+delta_minus #next nearest point
        if i+delta_minus>0:
            delta_minus+=-1
        else:
            # stop search in that direction 
            delta_minus=0
    elif d_plus!=-1 and (d_minus==-1 or d_minus>d_plus):
        i_next = i+delta_plus
        if i+delta_plus<nt-1:
            delta_plus+=1
        else:
            # stop search in that direction
            delta_plus=0
    else:  # nope (numba '0.56.3')
    #    # should never reach this point
    #    #assert False, (i, delta_minus, delta_plus, d_minus, d_plus)
    #    # AssertionError: (998, -1, 0, 0.9963861521472381, -1.0)
        print(('WARNING : pb advance search', i, delta_minus, delta_plus, d_minus, d_plus)) #ok numba 0.56.3
    return i_next, delta_minus, delta_plus
    
@njit
def find_nearest_neighboors(time, t, i):
    """ Find 3 remaining neighbouring points
    i is a starting value (closest point)
    """
    nt = len(time)
    nb = 4
    ib = [0 for _ in range(nb)]
    ib[0] = i
    #initiate direction for advance search (with problem at boundaries solved)
    if i==nt: #end-boundary case
        delta_plus=0
    else:
        delta_plus=1
    if i==0:
        delta_minus=0 # starting boundary case
    else:
        delta_minus=-1 # if not at boundaries delta_minus=-1 and delta_plus=1
    counter = 1
    while counter<nb:
        ib[counter], delta_minus, delta_plus = advance_search(nt, time, t, i, delta_plus, delta_minus)    
        counter+=1
    return np.sort(np.array(ib))

#@guvectorize([(float64[:], float64[:])], '(n)->(n)')
#@guvectorize(["void(float64[:], float64[:])"], '(n)->(n)')
# guvectorize cannot be called from a jit method at the moment, see: https://github.com/numba/numba/issues/5720
#def I_func(v, res):
@njit
def I_func(v):
    I = np.zeros_like(v)
    for i in range(v.shape[0]):
        if v[i]>-1 or v[i]<1:
            I[i] = v[i]
        else:
            I[i] = 0.
    return I

@njit
def solve_position_velocity(t_nb, x_nb, time_target):
    # solve for x and u :  x + u*(t_nb-date_target) = x_nb
    t = t_nb - time_target
    dt = t_nb[-1]-t_nb[0]
    weights = 70/81*(1-np.abs(t/dt)**3)**3 * I_func( t/dt )
    w = np.sum(weights)
    wt = np.sum(weights * t)
    wt2 = np.sum(weights * t**2)
    A = np.array([[w,wt],[wt,wt2]])#coef gradients
    b = np.array([np.sum(weights*x_nb), np.sum(weights*x_nb*t)])
    out = np.linalg.solve(A, b)
    return out[0], out[1]

@njit
def solve_position_velocity_acceleration(t_nb, x_nb, time_target):
    # solve for x and u :  x + u*(t_nb-date_target) = x_nb
    t = t_nb - time_target
    dt = t_nb[-1]-t_nb[0]
    weights = 70/81*(1-np.abs(t/dt)**3)**3 * I_func( t/dt )
    w = np.sum(weights)
    wt = np.sum(weights * t)
    wt2 = np.sum(weights * t**2)
    wt3 = np.sum(weights * t**3)
    wt4 = np.sum(weights * t**4)
    A = np.array([[w,wt, wt2],[wt,wt2, wt3], [wt2,wt3, wt4]])#coef gradients
    b = np.array([np.sum(weights*x_nb), np.sum(weights*x_nb*t), np.sum(weights*x_nb*t**2)])
    out = np.linalg.solve(A, b)
    return out[0], out[1], out[2]

#@njit("UniTuple(float64[:], 2)(float64[:], float64[:], float64[:])")
@njit
def lowess(time, x, time_target, degree):
    """ perform a lowess interpolation
    
    Parameters
    ----------
    time: np.array
        time array, assumed to be sorted in time, should be floats
    x: np.array
        positions
    time_target: np.array
        target timeline
    degree : 2 or 3, of the polynomial
    """
    nt = len(time_target)
    
    assert time_target[0]>=time[0], "time_target[0] is not within time span"
    assert time_target[-1]<=time[-1], "time_target[-1] is not within time span"
        
    # find closest values
    #d = np.abs(time[:,None] - time_target[None, :]) # nope (numba '0.56.3')
    d = np.abs(time.reshape(len(time), 1) -time_target.reshape(1, nt))

    i_closest = np.argmin(d, axis=0)#the indice of the nearest time in the raw time series (time) for each time of the regular time series (time_target)

    x_out = np.zeros(nt)
    u_out = np.zeros(nt)
    a_out = np.zeros(nt)
    
    for i in prange(nt):
        i_nb = find_nearest_neighboors(time, time_target[i], i_closest[i])
        t_nb = time[i_nb]
        x_nb = x[i_nb]
        if degree == 2:
            try : 
                x_out[i], u_out[i] = solve_position_velocity(t_nb, x_nb, time_target[i])
            except : print('WARNING :  pb with solve_position_velocity')  
        elif degree ==3 :
            try :
                x_out[i], u_out[i], a_out[i] = solve_position_velocity_acceleration(t_nb, x_nb, time_target[i])
            except : print('WARNING :  pb with solve_position_velocity')   

    return x_out, u_out, a_out
    
def lowess_smooth (df, t_target, degree, import_columns = None, geo=False, acc=False):
    """ perform a lowess interpolation
    
    Parameters
    ----------
    df: dataframe, must contain x, y
    t_target: `pandas.core.indexes.datetimes.DatetimeIndex` or str
                Output time series, as typically given by pd.date_range or the delta time of the output time series as str
                In this case, t_target is then recomputed taking start-end the start end of the input trajectory and the given delta time 
    degree : 2 or 3, degree of the polynomial for the lowess method
    import_columns : list of str
        list of df constant columns we want to import (ex: id, platform)    
    geo: boolean,
        optional if geo obj with projection
    acc: boolean,
        optional compute acceleration
    Return : dataframe with x, y, u, v, ax-ay computed from xy, au-av computed from u-v, and ae-an computed via lowess if degree = 3,+norms, id, platform
    """
    #index = time
    if df.index.name!='time':
        if df.index.name == None:
            df = df.set_index('time')
        else : 
            df =df.reset_index().set_index('time')  
    
    # assert x, y in dataframe
    if 'x' not in df or 'y' not in df :
        assert False, "positions must be labelled as 'x' and 'y'"
        
    # store projection to align with dataframes produced
    if geo :
        proj_ref = df.geo.projection_reference
    
    # time in seconds from first time
    df['date'] = (df.index-df.index.min())/pd.Timedelta('1s')
    
    #t_target
    if isinstance(t_target, str) :
        t_target = pd.date_range(df.index.min(), df.index.max(), freq = t_target)
    #t_ target in seconds from first time
    date_target = (t_target-t_target[0])/pd.Timedelta('1s')
    
    #apply lowess
    x_out, u_out, ax_out = lowess(df.date.values, df.x.values, date_target.values, degree=degree)
    y_out, v_out, ay_out = lowess(df.date.values, df.y.values, date_target.values, degree=degree)
    
    #dataframe
    if degree ==2 :
        df_out = pd.DataFrame(dict(x=x_out, y=y_out,u=u_out, v=v_out, time=t_target))
    if degree ==3 :
        df_out = pd.DataFrame(dict(x=x_out, y=y_out, u=u_out, v=v_out, ae=ax_out, an=ay_out, time=t_target))
        df_out['aen'] = np.sqrt(df_out.ae**2 +df_out.an**2)
    
    df_out['uv'] = np.sqrt(df_out.u**2 +df_out.v**2)
    
    #import columns/info ex: id or time
    if import_columns :
        for column in import_columns :
             df_out[column] = df[column][0]     
    
    # update lon/lat
    if geo:
        df_out['lon'] = df.lon.mean()
        df_out['lat'] = df.lat.mean()
        # first reset reference from df
        df_out.geo.set_projection_reference(proj_ref)  # inplace
        df_out.geo.compute_lonlat()  # inplace
    
    df_out =df_out.set_index('time')
    
    # recompute acceleration
    if acc:
        if geo :
            df_out.geo.compute_accelerations(
                from_ = ("xy", "x", "y"),
                names = ("ax", "ay", "axy"),
                centered_velocity=True,
                time='index',
                fill_startend=False,
                inplace=True,
            )
            df_out.geo.compute_accelerations(
                from_ = ("velocities", "u", "v"),
                names =("au", "av", "auv"),
                centered_velocity=True,
                time='index',
                fill_startend=False,
                inplace=True,
            ) 
            # should still recompute for non-geo datasets
        else:
            compute_accelerations(
                df_out,
                from_ = ("xy", "x", "y"),
                names = ("ax", "ay", "axy"),
                centered_velocity=True,
                time='index',
                fill_startend=False,
                inplace=True,
                keep_dt=False
            )
            compute_accelerations(
                df_out,
                from_ = ("velocities", "u", "v"),
                names =("au", "av", "auv"),
                centered_velocity=True,
                time='index',
                fill_startend=False,
                inplace=True,
                keep_dt=False
            )     
        
    return df_out

In [23]:
df_lowess= lowess_smooth(df_.reset_index(), '60min', degree=3, import_columns = ['id','platform'], geo=True, acc=True)
df_lowess

,x,y,u,v,ae,an,aen,uv,id,platform,lon,lat,ax,ay,axy,au,av,auv
time,,,,,,,,,,,,,,,,,,
2023-04-23 22:04:58,-152814.156300,259182.659759,-377.828038,117.557502,2.097931e-01,-0.065170,0.219682,395.694064,0-4388553,carthe_cnr_00,6.010115,40.472245,NaN,NaN,NaN,NaN,NaN,NaN
2023-04-23 23:04:58,-303557.232946,306625.155978,0.149446,0.156863,-6.337502e-06,0.000027,0.000028,0.216657,0-4388553,carthe_cnr_00,4.211810,40.858937,1.166350e-02,-0.003621,0.012213,5.248554e-02,-1.630512e-02,0.054960
2023-04-24 00:04:58,-303141.334306,307143.613882,0.067851,0.160621,-1.459949e-05,0.000014,0.000020,0.174364,0-4388553,carthe_cnr_00,4.216490,40.863748,-2.129170e-06,0.000003,0.000004,-7.276007e-06,-9.067327e-07,0.000007
2023-04-24 01:04:58,-302753.029712,307699.051077,0.097059,0.150335,-2.637436e-06,0.000003,0.000004,0.178944,0-4388553,carthe_cnr_00,4.220825,40.868882,-9.885818e-06,0.000003,0.000010,-3.717393e-06,2.417597e-06,0.000004
2023-04-24 02:04:58,-302492.845315,308293.727762,0.041086,0.178028,-1.813488e-05,-0.000002,0.000018,0.182707,0-4388553,carthe_cnr_00,4.223624,40.874323,-2.387652e-05,0.000002,0.000024,-1.714716e-05,3.148786e-06,0.000017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-15 21:04:58,158994.249728,-92775.482002,0.211308,0.050608,-2.606854e-06,0.000031,0.000031,0.217284,0-4388553,carthe_cnr_00,9.605112,37.301674,-7.404869e-08,0.000009,0.000009,2.102216e-06,-1.150311e-06,0.000002
2023-06-15 22:04:58,159768.318789,-92527.072570,0.217934,0.042045,6.233621e-07,-0.000015,0.000015,0.221953,0-4388553,carthe_cnr_00,9.613895,37.303777,4.199615e-06,-0.000005,0.000007,5.592626e-06,2.300180e-06,0.000006
2023-06-15 23:04:58,160596.814854,-92343.180097,0.251575,0.067170,2.428887e-05,0.000020,0.000031,0.260387,0-4388553,carthe_cnr_00,9.623278,37.305289,2.669796e-06,0.000012,0.000012,3.724626e-06,7.688909e-06,0.000009


In [24]:
hvplot = (
    (df_lowess.x.hvplot(label ='lowess') 
     * df_.set_index('time').x.hvplot(label = 'raw')
    )
    +(
        df_lowess.lon.hvplot(label ='lowess') 
      * df_.set_index('time').lon.hvplot(label = 'raw')
     ))

layout = hv.Layout(hvplot).cols(1)
layout

:Layout
   .Overlay.I  :Overlay
      .Curve.Lowess :Curve   [time]   (x)
      .Curve.Raw    :Curve   [time]   (x)
   .Overlay.II :Overlay
      .Curve.Lowess :Curve   [time]   (lon)
      .Curve.Raw    :Curve   [time]   (lon)

In [25]:
hvplot = (
    (df_lowess.y.hvplot(label ='lowess') 
     * df_.set_index('time').y.hvplot(label = 'raw')
    )
    +(
        df_lowess.lat.hvplot(label ='lowess') 
      * df_.set_index('time').lat.hvplot(label = 'raw')
     ))

layout = hv.Layout(hvplot).cols(1)
layout

:Layout
   .Overlay.I  :Overlay
      .Curve.Lowess :Curve   [time]   (y)
      .Curve.Raw    :Curve   [time]   (y)
   .Overlay.II :Overlay
      .Curve.Lowess :Curve   [time]   (lat)
      .Curve.Raw    :Curve   [time]   (lat)

In [70]:
hvplot = (
    (df_lowess.u.hvplot(label ='lowess') 
     * df_.set_index('time').velocity_east.hvplot(label = 'raw')
    )
    +(
        df_lowess.v.hvplot(label ='lowess') 
      * df_.set_index('time').velocity_north.hvplot(label = 'raw')
     ))

layout = hv.Layout(hvplot).cols(1)
layout

:Layout
   .Overlay.I  :Overlay
      .Curve.Lowess :Curve   [time]   (u)
      .Curve.Raw    :Curve   [time]   (velocity_east)
   .Overlay.II :Overlay
      .Curve.Lowess :Curve   [time]   (v)
      .Curve.Raw    :Curve   [time]   (velocity_north)